In [ ]:
%pylab inline
import pandas as pd
import xarray as xr
import geopandas as gp
from functools import reduce
from jupyterthemes import jtplot
import hyeenna as hy
jtplot.style(jtplot.infer_theme(), fscale=1.8)
jtplot.figsize(x=16, y=10)

wb_vars = ['precipitation', 'swe', 'evaporation', 'runoff', 'soil_moisture']

In [ ]:
plt.plot([0])
jtplot.figsize(x=10, y=8)
plt.close()

In [ ]:
def subtract_yearly_min(da):
    return (da.groupby(da.time.dt.year)
            .apply(lambda x: x - x.min(dim='time', skipna=True))
            .drop('year'))

def fix(ds):
    ds *= -1
    ds['precipitation'] *= -1
    return ds

def raw_data(data_array):
    return data_array.values.flatten()

def get_data(ds, dvars=True):
    evap     = raw_data(ds['evaporation'])[1:]
    runoff   = raw_data(ds['runoff'])[1:]
    precip   = raw_data(ds['precipitation'])[1:]
    if dvars:
        soil_liq = raw_data(ds['soil_moisture'].diff(dim='time'))
        swe      = raw_data(ds['swe'].diff(dim='time'))
        names = ['R', 'ET', 'P', 'ΔSM', 'ΔSWE']
    else:
        soil_liq = raw_data(ds['soil_moisture'])[1:]
        swe      = raw_data(ds['swe'])[1:]
        names = ['R', 'ET', 'P', 'SM', 'SWE']
    varlist = [runoff, evap, precip, soil_liq, swe]
    varlist = [((v - v.mean()) / v.std()).flatten() for v in varlist]
    varlist = [v.flatten() for v in varlist]
    #varlist = [v[::-1] for v in varlist]
    return names, varlist

In [ ]:
summa_will = fix(xr.open_dataset('./data/summa_will_info.nc'))
summa_will['swe'] = subtract_yearly_min(summa_will['swe'])
summa_will['soil_moisture'] = subtract_yearly_min(summa_will['soil_moisture'])

summa_snake = fix(xr.open_dataset('./data/summa_snake_info.nc'))
summa_snake['swe'] = subtract_yearly_min(summa_snake['swe'])
summa_snake['soil_moisture'] = subtract_yearly_min(summa_snake['soil_moisture'])

summa_rockies = fix(xr.open_dataset('./data/summa_rockies_info.nc'))
summa_rockies['swe'] = subtract_yearly_min(summa_rockies['swe'])
summa_rockies['soil_moisture'] = subtract_yearly_min(summa_rockies['soil_moisture'])

summa_olys = fix(xr.open_dataset('./data/summa_olys_info.nc'))
summa_olys['swe'] = subtract_yearly_min(summa_olys['swe'])
summa_olys['soil_moisture'] = subtract_yearly_min(summa_olys['soil_moisture'])

vic_will = xr.open_dataset('./data/vic_will_info.nc')
vic_will['swe'] = subtract_yearly_min(vic_will['swe'])
vic_will['soil_moisture'] = subtract_yearly_min(vic_will['soil_moisture'])

vic_snake = xr.open_dataset('./data/vic_snake_info.nc')
vic_snake['swe'] = subtract_yearly_min(vic_snake['swe'])
vic_snake['soil_moisture'] = subtract_yearly_min(vic_snake['soil_moisture'])

vic_rockies = xr.open_dataset('./data/vic_rockies_info.nc')
vic_rockies['swe'] = subtract_yearly_min(vic_rockies['swe'])
vic_rockies['soil_moisture'] = subtract_yearly_min(vic_rockies['soil_moisture'])

vic_olys = xr.open_dataset('./data/vic_olys_info.nc')
vic_olys['swe'] = subtract_yearly_min(vic_olys['swe'])
vic_olys['soil_moisture'] = subtract_yearly_min(vic_olys['soil_moisture'])

prms_will = xr.open_dataset('./data/prms_will_info.nc')
prms_will['swe'] = subtract_yearly_min(prms_will['swe'])
prms_will['soil_moisture'] = subtract_yearly_min(prms_will['soil_moisture'])

prms_snake = xr.open_dataset('./data/prms_snake_info.nc')
prms_snake['swe'] = subtract_yearly_min(prms_snake['swe'])
prms_snake['soil_moisture'] = subtract_yearly_min(prms_snake['soil_moisture'])

prms_rockies = xr.open_dataset('./data/prms_rockies_info.nc')
prms_rockies['swe'] = subtract_yearly_min(prms_rockies['swe'])
prms_rockies['soil_moisture'] = subtract_yearly_min(prms_rockies['soil_moisture'])

prms_olys = xr.open_dataset('./data/prms_olys_info.nc')
prms_olys['swe'] = subtract_yearly_min(prms_olys['swe'])
prms_olys['soil_moisture'] = subtract_yearly_min(prms_olys['soil_moisture'])

In [ ]:
analysis_dict = {
    'summa_will': summa_will,
    'summa_snake': summa_snake,
    'summa_rockies': summa_rockies,
    'summa_olys': summa_olys,
    'vic_will': vic_will,
    'vic_snake': vic_snake,
    'vic_rockies': vic_rockies,
    'vic_olys': vic_olys,
    'prms_will': prms_will,
    'prms_snake': prms_snake,
    'prms_rockies': prms_rockies,
    'prms_olys': prms_olys
}
season = ('JJA', 'MAM', 'DJF', 'SON')
for k, v in analysis_dict.items():
    print(k)
    analysis_dict[k] = v#.where(v.time.dt.season=='SON', drop=True)#resample(time='W').mean()
    #sub = v.resample(time='W').sum(dim='time')
    #sub['swe'].values = v['swe'].resample(time='W').mean(dim='time')
    #sub['soil_moisture'].values = v['soil_moisture'].resample(time='W').mean(dim='time')
    #analysis_dict[k] = sub


In [ ]:
name = 'summa_will'
ds = analysis_dict[name]
names, varlist = get_data(ds, dvars=True)
df = hy.estimate_info_transfer_network(
                         varlist, names,  condition=False,
                         tau=1, omega=1, nu=1, k=1, l=1, m=1,
                         sample_size=5000, nruns=30)
df.to_csv('./data/network_data/{}.csv'.format(name))

In [ ]:
colors = [matplotlib.colors.to_hex(plt.cm.Dark2(i), keep_alpha=False) 
          for i in np.linspace(0, 1, len(names))] 
colors = ['#83a83b', '#3472c6', '#c44e52', '#8172b2', '#ff914d']
np.fill_diagonal(df.values, 0)
hy.plot_chords(df.values.T, names, colors, 0.85) #[0.3, 0.3, 0.3, 0.9, 0.3])